In [9]:
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mc
import colorsys
import time 

Color Intensity setting

In [10]:
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])
    

Setting up functions to find distance and unit vectors

In [11]:
def distance(p1, p2):
    """
    Gives us the distance between two points in space
    
    Args:
        p1 : point 1 in space, NumPy array
        p2 : point 2 in space, NumPy array
    """
    return np.linalg.norm(p2-p1)

def normalise(vector):
    """
    Gives you the normal vector for a given vector
    
    Args:
        vector : The vector that needs to be normalised
    """
    return vector/(np.linalg.norm(vector)) # Unit vector in a certain direction 

Setting up the FABRIK Function

In [12]:
num_link = 4
reps=300000


def positions(phi,l=2*np.ones((num_link,))):
    """
    Find the position of each joint of the mechanism
    Args:
        phi : The angles of the joints with respect to the positive x axis
        l : Length of the links
    """
    x, y=np.zeros((num_link+1,1)), np.zeros((num_link+1,1))
     
    for i in range(1, num_link+1):
        x[i,:], y[i,:]=x[i-1,:]+l[i-1]*np.cos(phi[i-1]), y[i-1,:]+l[i-1]*np.sin(phi[i-1])
    
    return np.concatenate((x,y),axis=1)



p_dash = np.zeros((reps,num_link+1,2))

for i in range(0,reps):
    alpha=np.random.uniform(-np.pi,np.pi,(num_link,))
    p_dash[i,:,:]=positions(phi=alpha)

# print(p_dash[:,4,:].shape)


In [13]:

def FABRIK_solver(aim,p=p_dash,num_links = num_link,length=2*np.ones((num_link,)),start=np.array([[0,0]]),max_iterations=10,epsilon=1e-3):
    """
    Uses the FABRIK technique to solve for a planar manipulator, for multiple different initial configurations.
    
    Args:
        start : The position of the starting point, 1x2 matrix that is broadcasted to repx2 shape.
        aim : The desired position of the end effector 
        4 : Number of links  
        length : Length of links
        max_iterations : The maximum number of iterations of the algorithm before deciding on outcome, default is 100
        epsilon : The maximum error that can be allowed between the estimated end effector 
                  position and the actual end effector postion, default is 1e-3
    """

    # Initialising  
    thetas = np.zeros((reps,1,num_links))

    #Checking to see if the point is in the range of the manipulator 
    if np.sum(length)<distance(aim,start):
        print("This point cannot be reached")
        exit()

    for iter in range(max_iterations):  
        error = np.zeros((reps,))
        for rep in range(0,reps):
            error[rep] = distance(aim, p[rep,num_links,:])
            
        if (error>=epsilon).all:    
            
            #Initialise the goal values 
            target = np.repeat(aim,reps,axis=0)
            fixed = np.repeat(start,reps,axis=0)
            
            #Array of angles initialised 
            phi = np.zeros((reps,))
            vect = np.zeros((reps,2))
            unit_vect = np.zeros((reps,2))
            
            for link in range(num_links,0,-1):
                for r in range(0,reps):
                    vect[r,:] = p[r,link-1,:]-target[r,:]
                    unit_vect[r,:] = normalise(vect[r,:])
                    phi[r] = np.arctan2(unit_vect[r,1],unit_vect[r,0])
                    
                p[:,link,:]=target
                p[:,link-1,:] = target+length[link-1]*np.transpose((np.array([np.cos(phi[:]), np.sin(phi[:])])))
                target=p[:,link-1,:]
                thetas[:,:,link-1]=np.reshape(np.arctan2(p[:,link,1]-p[:,link-1,1],p[:,link,0]-p[:,link-1,0]).reshape((reps,1)), (reps,1))

            alpha = np.zeros((reps,))
            vecto=np.zeros((reps,2))
            norm_vecto=np.zeros((reps,2))

            
            for link in range(0,num_links):
                for r in range(0,reps):
                    vecto[r,:] = p[r,link+1,:]-fixed[r,:]
                    norm_vecto[r,:] = normalise(vecto[r,:])
                    alpha[r] = np.arctan2(norm_vecto[r,1],norm_vecto[r,0])
                p[:,link,:]=fixed
                p[:,link+1,:]=fixed+length[link]*np.transpose((np.array([np.cos(alpha[:]),np.sin(alpha[:])])))
                fixed=p[:,link+1,:]
        
                thetas[:,:,link]=np.reshape(np.arctan2(p[:,link+1,1]-p[:,link,1],p[:,link+1,0]-p[:,link,0]),(reps,1))

    return thetas,p
                

Testing the code

In [14]:


# final=np.array([[-1.5,-0.8]])

# start_time = time.time()
# angles,points = FABRIK_solver(aim=final)
# end_time = time.time()
# time_taken = end_time-start_time

# print(time_taken)
# # print(angles, "\nis the array of angles")
# # print()
# # print(points, "\nis the array of points")

# plt.figure(dpi=150)
# for i in range(0,reps):
#     plt.plot(points[i,:,0],points[i,:,1],"-",color=lighten_color("k",0.5),linewidth="0.5",marker="o", label="Configuration")
# plt.plot(final[:,0],final[:,1],marker="o",color="k", label="Target End Effector position")
# plt.plot(0,0,marker="o",color="r", label="Start Point")
# # plt.plot(final[0],final[1],marker="^",color="b")
# plt.xlim(-4,4)
# plt.ylim(-4,4)
# # plt.legend()
# plt.grid()
# plt.show()



# print(distance(points[0],points[1]))
# print(distance(points[1],points[2]))
# print(distance(points[2],points[3]))
# print(distance(points[3],points[4]))

In [15]:
def fabrik(aim,p=p_dash[0,:,:],length=np.array([2,2,2,2]),start=np.array([0,0]),max_iterations=10,epsilon=1e-3):
    """
    Uses the FABRIK technique to solve for a planar manipulator, when solving one by one for each initial configuration
    
    Args:
        start : The position of the starting point
        aim : The desired position of the end effector 
        4 : Number of links  
        length : Length of links
        max_iterations : The maximum number of iterations of the algorithm before deciding on outcome, default is 100
        epsilon : The maximum error that can be allowed between the estimated end effector 
                  position and the actual end effector postion, default is 1e-3
    """
    # Initialising 

    thetas = np.zeros((1,4))
    if np.sum(length)<distance(aim,start):
        print("This point cannot be reached")
#       break

    else:
        if np.abs(aim-p[4,:]).all()>=epsilon:            
            for iter in range(max_iterations):
                #Initialise the goal values 
                target = aim
                fixed = start

                for link in range(4,0,-1):
                    vect = p[link-1,:]-target
                    unit_vect = normalise(vect)
                    phi = np.arctan2(unit_vect[1],unit_vect[0])
                    p[link,:]=target
                    p[link-1,:] = target+length[link-1]*(np.array([np.cos(phi), np.sin(phi)]))
                    target=p[link-1,:]
                    thetas[:,link-1]=np.arctan2(p[link,1]-p[link-1,1],p[link,0]-p[link-1,0])

                for link in range(0,4):
                    vecto = p[link+1,:]-fixed
                    norm_vecto = normalise(vecto)
                    alpha = np.arctan2(norm_vecto[1],norm_vecto[0])
                    p[link,:]=fixed
                    p[link+1,:]=fixed+length[link]*(np.array([np.cos(alpha),np.sin(alpha)]))
                    fixed=p[link+1,:]
                    thetas[:,link]=np.arctan2(p[link+1,1]-p[link,1],p[link+1,0]-p[link,0])

    return thetas,p


In [16]:
# final=np.array([-1.5,-0.8])

# starting = time.time()
# for i in range(0,reps):
#     angles,points = fabrik(aim=final,p=p_dash[i,:,:])
# ending = time.time()

# timing_og = ending - starting
# print(timing_og)

# print(angles, "\nis the array of angles")
# print()
# print(points, "\nis the array of points")

# plt.figure(dpi=150)
# plt.plot(points[:,0],points[:,1],"-",color=lighten_color("k",0.5),linewidth="0.5",marker="o", label="Configuration")
# plt.plot(points[4,0],points[4,1],marker="o",color="k", label="Target End Effector position")
# plt.plot(0,0,marker="o",color="r", label="Start Point")
# # plt.plot(final[0],final[1],marker="^",color="b")
# plt.xlim(-4,4)
# plt.ylim(-4,4)
# plt.legend()
# plt.grid()
# plt.show()

# print(distance(points[0],points[1]))
# print(distance(points[1],points[2]))
# print(distance(points[2],points[3]))
# print(distance(points[3],points[4]))